In [35]:
def calculate_tree(elements):
    if len(elements) != 8:
        raise ValueError("The list must contain exactly 8 elements.")
    
    # Create the tree nodes
    x = elements
    
    # Calculate the intermediate nodes
    x_1 = x[0] * x[1]
    x_2 = x[2] * x[3]
    x_3 = x[4] * x[5]
    x_4 = x[6] * x[7]
    
    # Calculate the next level of nodes
    x_5 = x_1 * x_2
    x_6 = x_3 * x_4
    
    # Calculate the root node
    ROOT = x_5 * x_6
    
    # Return the calculated tree
    return [x_1, x_2, x_3, x_4, x_5, x_6, ROOT]

p = 120.next_prime()
F = GF(p)
R = PolynomialRing(F, 'x')
x = R.gen()

child_nodes = [F(x) for x in [87, 71, 59, 88, 55, 102, 2, 109]]
tree = calculate_tree(child_nodes)

g = enumerate(child_nodes + tree)
low_degree_extension = R.lagrange_polynomial(g)
low_degree_extension

23*x^14 + 82*x^13 + 27*x^12 + 74*x^11 + 45*x^10 + 29*x^9 + 59*x^8 + 24*x^7 + 16*x^6 + 6*x^5 + 37*x^4 + 73*x^3 + 95*x^2 + 29*x + 87

In [40]:
def input(j):
    assert 8 <= j <= 14
    if j == 8: return (0, 1)
    elif j == 9: return (2, 3)
    elif j == 10: return (4, 5)
    elif j == 11: return (6, 7)
    elif j == 12: return (8, 9)
    elif j == 13: return (10, 11)
    elif j == 14: return (12, 13)

input_l = R.lagrange_polynomial([(i, input(i)[0]) for i in range(8, 15)])
print(f"input left: {input_l}")
input_r = R.lagrange_polynomial([(i, input(i)[1]) for i in range(8, 15)])
print(f"input left: {input_r}")


h = lambda x: low_degree_extension(x) - low_degree_extension(input_l(x)) * low_degree_extension(input_r(x))
assert all([h(i) == 0 for i in range(8, 15)])

h_i = R.lagrange_polynomial(enumerate([h(i) for i in range(15)]))
print(f"h_i: {h_i}")

vanishing_poly = prod([(x - i) for i in range(8, 15)])
print(f"vanishing polynomial: {vanishing_poly}")

input left: 2*x + 111
input left: 2*x + 112
h_i: 71*x^14 + 117*x^13 + 15*x^12 + 69*x^11 + 29*x^10 + 13*x^9 + 17*x^8 + 15*x^7 + 92*x^6 + 70*x^5 + 116*x^4 + 92*x^3 + x^2 + 72*x + 72
vanishing polynomial: x^7 + 50*x^6 + 114*x^5 + 32*x^4 + 117*x^3 + 4*x^2 + 3*x + 120


In [41]:
quotient, remainder = h_i.quo_rem(banishing_poly)
quotient, remainder

(71*x^7 + 123*x^6 + 122*x^5 + 27*x^4 + 87*x^3 + 41*x^2 + 19*x + 26, 0)

In [27]:
# Import necessary libraries
from sage.all import *

# Initialize the list of 8 elements
elements = [1, 2, 3, 4, 5, 6, 7, 8]

# Calculate the values in the binary tree
def calculate_tree(elements):
    if len(elements) != 8:
        raise ValueError("The list must contain exactly 8 elements.")
    
    # Create the tree nodes
    x = elements
    
    # Calculate the intermediate nodes
    x_1 = x[0] * x[1]
    x_2 = x[2] * x[3]
    x_3 = x[4] * x[5]
    x_4 = x[6] * x[7]
    
    # Calculate the next level of nodes
    x_5 = x_1 * x_2
    x_6 = x_3 * x_4
    
    # Calculate the root node
    ROOT = x_5 * x_6
    
    # Return the calculated tree
    return x + [x_1, x_2, x_3, x_4, x_5, x_6, ROOT]

# Calculate the tree values
tree_values = calculate_tree(elements)

# Enumerate the values
enumerated_values = list(enumerate(tree_values))

# Create the field
F = GF(17)  # Example with a finite field of order 17

# Interpolation points
points = [(F(i), F(v)) for i, v in enumerated_values]

# Create the Lagrange polynomial
P = PolynomialRing(F, 'x', mod = )
x = P.gen()
L = P.lagrange_polynomial(points)

# Print the Lagrange polynomial
print("Lagrange Polynomial:")
print(L)

# Get the vanishing polynomial from the interpolation points
def vanishing_polynomial(points):
    z = 1
    for a, _ in points:
        z *= (x - a)
    return z

# Calculate the vanishing polynomial
Z = vanishing_polynomial(points)

# Print the vanishing polynomial
print("Vanishing Polynomial:")
print(Z)

# Calculate the quotient polynomial
quotient, remainder = L.quo_rem(Z)

# Print the quotient polynomial
print("Quotient Polynomial:")
print(quotient)

# Print the remainder to ensure it is zero
print("Remainder:")
print(remainder)


Lagrange Polynomial:
5*x^14 + 9*x^13 + 15*x^12 + 5*x^11 + 3*x^10 + 10*x^9 + 8*x^8 + 9*x^7 + 7*x^6 + 5*x^5 + 5*x^4 + 9*x^3 + 13*x^2 + 1
Vanishing Polynomial:
x^15 + 14*x^14 + 7*x^13 + 2*x^12 + 14*x^11 + 5*x^10 + 8*x^9 + x^7 + 14*x^6 + 7*x^5 + 2*x^4 + 14*x^3 + 5*x^2 + 8*x
Quotient Polynomial:
0
Remainder:
5*x^14 + 9*x^13 + 15*x^12 + 5*x^11 + 3*x^10 + 10*x^9 + 8*x^8 + 9*x^7 + 7*x^6 + 5*x^5 + 5*x^4 + 9*x^3 + 13*x^2 + 1


In [32]:
# Import necessary libraries
from sage.all import *

# Finite field of order 17
F = GF(17)

# List of 8 elements
elements = [1, 2, 3, 4, 5, 6, 7, 8]

# Binary tree multiplication (to simulate a function output)
def calculate_tree(elements):
    if len(elements) != 8:
        raise ValueError("The list must contain exactly 8 elements.")
    
    x = elements
    x_1 = x[0] * x[1]
    x_2 = x[2] * x[3]
    x_3 = x[4] * x[5]
    x_4 = x[6] * x[7]
    x_5 = x_1 * x_2
    x_6 = x_3 * x_4
    ROOT = x_5 * x_6
    
    return x + [x_1, x_2, x_3, x_4, x_5, x_6, ROOT]

# Calculate the tree values
tree_values = calculate_tree(elements)

# Interpolation points
points = [(F(i), F(v)) for i, v in enumerate(tree_values)]

# Create the polynomial ring
P = PolynomialRing(F, 'x')
x = P.gen()

# Create the Lagrange polynomial
L = P.lagrange_polynomial(points)

# Print the Lagrange polynomial
print("Lagrange Polynomial:")
print(L)


# Get the vanishing polynomial from the interpolation points
def vanishing_polynomial(points):
    z = 1
    for a, _ in points:
        z *= (x - a)
    return z

# Calculate the vanishing polynomial
Z = vanishing_polynomial(points)

# Print the vanishing polynomial
print("Vanishing Polynomial:")
print(Z)

# Calculate the quotient polynomial
quotient, remainder = L.quo_rem(Z)

# Print the quotient polynomial
print("Quotient Polynomial:")
print(quotient)

# Print the remainder to ensure it is zero
print("Remainder:")
print(remainder)

# Example polynomial to be used in PIOP SNARK (f is an example)
f = L  # In a real SNARK, this would be a function that computes the witness polynomial

# Evaluate polynomial f at some point
evaluation_point = F(10)
f_evaluation = f(evaluation_point)
print(f"Evaluation of f at {evaluation_point}: {f_evaluation}")

# Proof for the evaluation using the quotient polynomial
quotient_proof = quotient(evaluation_point)
print(f"Quotient Polynomial at {evaluation_point}: {quotient_proof}")

# Verify the proof by recomputing and checking the value
recomputed_value = Z(evaluation_point) * quotient_proof 
assert recomputed_value == f_evaluation
print(f"Verification successful: {recomputed_value} == {f_evaluation}")


Lagrange Polynomial:
5*x^14 + 9*x^13 + 15*x^12 + 5*x^11 + 3*x^10 + 10*x^9 + 8*x^8 + 9*x^7 + 7*x^6 + 5*x^5 + 5*x^4 + 9*x^3 + 13*x^2 + 1
Vanishing Polynomial:
x^15 + 14*x^14 + 7*x^13 + 2*x^12 + 14*x^11 + 5*x^10 + 8*x^9 + x^7 + 14*x^6 + 7*x^5 + 2*x^4 + 14*x^3 + 5*x^2 + 8*x
Quotient Polynomial:
0
Remainder:
5*x^14 + 9*x^13 + 15*x^12 + 5*x^11 + 3*x^10 + 10*x^9 + 8*x^8 + 9*x^7 + 7*x^6 + 5*x^5 + 5*x^4 + 9*x^3 + 13*x^2 + 1
Evaluation of f at 10: 13
Quotient Polynomial at 10: 0


AssertionError: 